<a href="https://colab.research.google.com/github/Filo2002/ImageRetrievalEsameElettrotecnica/blob/master/ampl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
%pip install -q amplpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.9 MB/s eta 0:00:00


In [2]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["minos", "cplex", "conopt", "baron", "gurobi", "snopt", "highs"],  # modules to install
    license_uuid="22d13bd9-fa62-4dae-8ed2-39fff6f7abed",  # license to use
)  # instantiate AMPL object and register magics

Licensed to Bundle #6252.6609 expiring 20240131: Ricerca Operativa, Prof. Fabrizio Marinelli, Universit? Politecnica delle Marche.


## sequenze in programmazione matematica

Alternative:
- $x_{ij} = 1$ se l'elemento $i$ precede l'elemento $j$ (non in senso stretto).
- $x_{ij} = 1$ se l'elemento $i$ è in posizione $j$.
- $x_{i} = j$ se l'elemento $i$ è in posizione $j$.

### Alternativa 1

$x_{ij} = 1$ se l'elemento $i$ precede l'elemento $j$ (non in senso stretto).


- proprietà antisimmetrica
$$
x_{ij} + x_{ji} = 1 \qquad \qquad i,j \in A, i \neq j     
$$

- proprietà transitiva
$$
x_{ij} + x_{jk} + x_{ki} \le 2 \qquad \qquad i,j,k \in A, i \neq j \neq k     
$$



In [4]:
%%ampl_eval
reset;

var xa integer;
var xb integer;
var xc integer;

s.t. vincolo1: xa + 2 * xb + 3 * xc <= 40;
s.t. vincolo2: 2 * xb - xa <= 0;
s.t. vincolo3: xc - xb <= 0;
s.t. vincolo4: xa >= 0;
s.t. vincolo5: xb >= 0;
s.t. vincolo6: xc >= 0;

maximize profitto: (xa - 2 * xb) * 8 + (xb - xc) * 70 + xc * 100;



#s.t. v2{i in A, j in A, k in A: i != j && j != k}: x[i,j] + x[j,k] + x[k,i] = 2;

In [5]:
%%ampl_eval
expand;

maximize profitto:
	8*xa + 54*xb + 30*xc;

subject to vincolo1:
	xa + 2*xb + 3*xc <= 40;

subject to vincolo2:
	-xa + 2*xb <= 0;

subject to vincolo3:
	-xb + xc <= 0;

subject to vincolo4:
	xa >= 0;

subject to vincolo5:
	xb >= 0;

subject to vincolo6:
	xc >= 0;



In [6]:
%%ampl_eval
option solver cplex;
option cplex_options "mipdisplay=2";
solve;

CPLEX 22.1.1.0: mipdisplay=2
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Reduced MIP has 2 rows, 2 columns, and 4 nonzeros.
Reduced MIP has 0 binaries, 2 generals, 0 SOSs, and 0 indicators.
Reduced MIP has 2 rows, 2 columns, and 4 nonzeros.
Reduced MIP has 0 binaries, 2 generals, 0 SOSs, and 0 indicators.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.01 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000      760.0000              --- 
*     0     0      integral     0      700.0000      700.0000        0    0.00%
Elapsed time = 0.02 sec. (0.01 ticks, tree = 0.00 MB)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time  

In [8]:
%%ampl_eval
display xa;
display xb;
display xc;

xa = 20

xb = 10

xc = 0



E' difficile leggere la sequenza ottima dalla visualizzazione diretta della matrice $\bf{x}$.

Conviene organizzare la soluzione in modo diverso osservando che il numero di 1 nella riga $i$-esima di $\bf{x}$ indica il numero di predecessori dell'elemento $i$.  

In [ ]:
%%ampl_eval
delete data index;
param index{A};
for{i in A} let index[i] := sum{j in A}x[j,i];

printf "\n\n La sequenza e': ";
for{i in A}{
    for{j in A}{
        if index[j] == i-1 then {printf "%i - ", j; break;}
    }
}

### Alternativa 2

$x_{ij} = 1$ se l'elemento $i$ è in posizione $j$.

- ogni posizione riceve **esattamente** un elemento:
$$
\sum_{i \in A} x_{ij} = 1 \qquad \qquad j \in A      
$$

- ogni elemento occupa **esattamente** una posizione:
$$
\sum_{j \in A} x_{ij} = 1 \qquad \qquad i \in A     
$$


In [ ]:
%%ampl_eval
reset;

param n := 10;
set A :=  {1..n};      # elelemnti
set P :=  {1..n};      # posizioni

# Dichiarazione delle variabili
var x{A,P} binary;

s.t. posizione{j in P}: sum{i in A} x[i,j] = 1;
s.t. elemento{i in A}: sum{j in P} x[i,j] = 1;

In [ ]:
%%ampl_eval
expand;

In [ ]:
%%ampl_eval
option solver cplex;
solve;

In [ ]:
%%ampl_eval
display x;

Di nuovo la sequenza ottima potrebbe essere difficile da interpretare direttamente dalla matrice $\bf{x}$.

In [ ]:
%%ampl_eval
delete data index;
param index{A};
for{j in A}{
    for{i in A}{
        if x[i,j] == 1 then {let index[j] := i; break;}
    }
}
printf "\n\n La sequenza e': ";
for{i in A}{ printf "%i - ", index[i]; }

### Alternativa 2b

$x_{i} = j$ se l'elemento $i$ è in posizione $j$.

Occorre modellare la condizione $ x_i \neq x_j$, ovvero $x_i < x_j$ **oppure** $x_i > x_j$.

- $x_i < x_j \implies s_{ij} = 1$
$$
x_{j} - x_{i} + 1 \le |A| s_{ij} \qquad \qquad i,j \in A, i \neq j      
$$
- $x_i > x_j \implies s_{ij} = 0$
$$
x_{i} - x_{j} + 1 \le |A| ( 1- s_{ij}) \qquad \qquad i,j \in A, i \neq j      
$$




In [ ]:
%%ampl_eval
reset;

param n := 10;
set A :=  {1..n};

# Dichiarazione delle variabili
var x{A} integer >= 1 <= n;
var s{A,A} binary;

s.t. v1{i in A, j in A: i != j}: x[j] - x[i] + 1 <= n*s[i,j];
s.t. v2{i in A, j in A: i != j}: x[i] - x[j] + 1 <= n*(1 - s[i,j]);

In [ ]:
%%ampl_eval
expand;
option solver cplex;
solve;

In [ ]:
%%ampl_eval
display x;

In [ ]:
%%ampl_eval
printf "\n\n La sequenza e': ";
for{j in A}{
    for{i in A}{
        if x[i] == j then {printf "%i - ", i; break;}
    }
}

### Qual è l'alternativa migliore?
- **Alternativa 1**: circa $n^2$ variabili binarie e circa $n^2 + n^3$ vincoli
- **Alternativa 2**: $n^2$ variabili binarie e $2n$ vincoli
- **Alternativa 2b**: $n$ variabili intere $n^2$ variabili binarie e $n^2$ vincoli

Utilizziamo una funzione obiettivo casuale per valutare le alternative.

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 1:  x_ij = 1 se i precede (non in senso stretto) j
reset;
param n;
set A :=  {1..n};
param c{A, A};

var x{A,A} binary;

minimize z: sum{i in A, j in A} c[i,j]*x[i,j];
s.t. v1{i in A, j in A: i != j}: x[i,j] + x[j,i] = 1;
s.t. v2{i in A, j in A, k in A: i != j && j != k}: x[i,j] + x[j,k] + x[k,i] <= 2;

data;
param n := 30;
param c :
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30 :=
1	4	23	43	91	57	1	63	47	74	68	15	84	10	28	35	18	53	46	88	67	97	67	91	52	71	75	100	95	37	44
2	16	96	97	50	63	15	22	80	37	16	96	31	57	38	100	13	63	29	43	79	80	47	43	12	15	13	82	97	68	70
3	94	74	50	83	22	70	66	20	100	88	36	47	86	21	21	58	72	84	53	75	100	43	51	93	59	52	55	58	9	16
4	60	33	64	73	81	56	34	74	97	12	57	68	94	85	94	13	88	24	100	77	12	39	3	89	81	49	18	31	94	83
5	11	77	53	30	8	3	78	63	78	2	34	63	65	41	24	56	51	36	2	79	11	82	8	96	1	66	75	16	62	86
6	19	77	23	37	81	78	26	92	12	40	17	74	72	36	19	67	5	57	89	65	17	69	40	8	67	10	71	87	97	40
7	97	84	77	31	51	93	51	95	73	8	92	83	14	73	51	63	91	44	73	48	71	75	76	24	55	42	6	80	56	20
8	92	84	93	38	28	38	33	7	21	100	43	23	6	44	68	52	53	19	73	12	95	1	65	62	27	22	71	6	50	36
9	87	3	10	89	67	78	77	90	72	37	49	6	59	25	84	57	80	26	31	95	83	68	97	47	2	56	42	29	45	47
10	25	37	10	14	41	25	41	65	49	40	67	45	78	47	25	96	2	4	40	65	86	9	44	17	57	42	77	17	59	62
11	6	10	38	90	70	72	74	86	97	82	45	28	97	81	72	26	98	76	64	79	28	30	93	27	95	64	84	53	70	72
12	21	33	90	74	22	74	96	94	50	69	69	54	19	26	24	24	57	10	88	25	66	42	52	28	38	44	89	18	38	32
13	90	48	47	89	34	20	79	69	11	99	57	3	60	89	19	55	99	61	89	66	19	6	85	9	38	52	15	27	37	46
14	76	46	1	71	73	13	23	5	92	20	90	90	55	16	21	46	30	56	14	13	93	33	34	28	9	81	91	35	41	79
15	4	53	50	43	46	42	66	22	30	30	58	4	88	83	44	21	92	38	46	25	48	65	36	29	71	92	99	1	78	76
16	38	54	79	1	78	72	4	83	70	85	67	66	72	86	95	96	32	89	56	60	74	8	53	25	13	97	78	24	6	22
17	81	9	96	19	40	72	1	66	52	80	85	24	17	87	98	21	9	63	63	32	26	57	93	31	78	87	91	37	4	93
18	3	98	25	9	35	57	7	16	65	38	56	43	54	44	55	6	33	56	85	4	48	40	4	44	37	81	59	56	40	96
19	36	7	20	93	11	1	27	69	64	60	75	73	40	72	37	69	90	60	75	93	74	87	76	4	71	72	31	14	47	62
20	32	56	16	17	76	51	31	44	31	96	34	23	3	96	93	61	35	81	28	35	98	57	41	22	70	79	96	84	8	82
21	54	66	13	78	15	70	69	33	53	6	9	94	35	79	64	20	68	81	60	87	50	55	34	66	56	58	58	17	36	28
22	33	80	10	45	75	18	76	51	2	28	100	5	48	5	43	74	99	75	32	95	80	20	26	40	63	48	11	15	65	28
23	54	91	88	61	69	48	73	12	64	7	71	45	17	66	18	82	91	74	53	96	77	20	29	54	35	92	74	10	93	18
24	43	91	32	92	11	5	65	85	19	77	100	63	16	15	44	95	11	71	56	15	4	40	16	34	62	82	88	11	53	11
25	6	64	32	42	73	18	23	73	94	24	76	43	39	66	54	33	69	70	45	23	64	77	61	6	63	27	78	58	10	41
26	94	53	73	14	74	69	87	75	62	95	46	44	78	40	63	2	50	49	46	94	57	49	24	92	87	75	7	25	46	49
27	4	85	71	44	76	85	92	40	59	37	98	97	19	61	39	54	6	55	51	28	33	77	53	84	84	95	50	37	39	88
28	95	29	5	74	15	44	29	97	36	7	58	14	43	11	63	18	80	84	75	76	83	82	12	21	21	5	2	39	63	61
29	77	54	18	17	24	32	3	65	32	91	62	64	74	18	35	75	90	94	20	79	3	79	61	25	39	69	40	52	78	74
30	82	86	30	89	100	80	35	68	94	22	84	27	48	28	44	12	13	28	81	36	8	83	5	28	20	18	46	34	98	18
;


option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 2:   x_ij = 1 se i è in posizione j
reset;
param n;
set A :=  {1..n};      # elelemnti
set P :=  {1..n};      # posizioni
param c{A, P};

# Dichiarazione delle variabili
var x{A,P} binary;

minimize z: sum{i in A, j in P} c[i,j]*x[i,j];
s.t. posizione{j in P}: sum{i in A} x[i,j] = 1;
s.t. elemento{i in A}: sum{j in P} x[i,j] = 1;



data;
param n := 30;
param c :
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30 :=
1	4	23	43	91	57	1	63	47	74	68	15	84	10	28	35	18	53	46	88	67	97	67	91	52	71	75	100	95	37	44
2	16	96	97	50	63	15	22	80	37	16	96	31	57	38	100	13	63	29	43	79	80	47	43	12	15	13	82	97	68	70
3	94	74	50	83	22	70	66	20	100	88	36	47	86	21	21	58	72	84	53	75	100	43	51	93	59	52	55	58	9	16
4	60	33	64	73	81	56	34	74	97	12	57	68	94	85	94	13	88	24	100	77	12	39	3	89	81	49	18	31	94	83
5	11	77	53	30	8	3	78	63	78	2	34	63	65	41	24	56	51	36	2	79	11	82	8	96	1	66	75	16	62	86
6	19	77	23	37	81	78	26	92	12	40	17	74	72	36	19	67	5	57	89	65	17	69	40	8	67	10	71	87	97	40
7	97	84	77	31	51	93	51	95	73	8	92	83	14	73	51	63	91	44	73	48	71	75	76	24	55	42	6	80	56	20
8	92	84	93	38	28	38	33	7	21	100	43	23	6	44	68	52	53	19	73	12	95	1	65	62	27	22	71	6	50	36
9	87	3	10	89	67	78	77	90	72	37	49	6	59	25	84	57	80	26	31	95	83	68	97	47	2	56	42	29	45	47
10	25	37	10	14	41	25	41	65	49	40	67	45	78	47	25	96	2	4	40	65	86	9	44	17	57	42	77	17	59	62
11	6	10	38	90	70	72	74	86	97	82	45	28	97	81	72	26	98	76	64	79	28	30	93	27	95	64	84	53	70	72
12	21	33	90	74	22	74	96	94	50	69	69	54	19	26	24	24	57	10	88	25	66	42	52	28	38	44	89	18	38	32
13	90	48	47	89	34	20	79	69	11	99	57	3	60	89	19	55	99	61	89	66	19	6	85	9	38	52	15	27	37	46
14	76	46	1	71	73	13	23	5	92	20	90	90	55	16	21	46	30	56	14	13	93	33	34	28	9	81	91	35	41	79
15	4	53	50	43	46	42	66	22	30	30	58	4	88	83	44	21	92	38	46	25	48	65	36	29	71	92	99	1	78	76
16	38	54	79	1	78	72	4	83	70	85	67	66	72	86	95	96	32	89	56	60	74	8	53	25	13	97	78	24	6	22
17	81	9	96	19	40	72	1	66	52	80	85	24	17	87	98	21	9	63	63	32	26	57	93	31	78	87	91	37	4	93
18	3	98	25	9	35	57	7	16	65	38	56	43	54	44	55	6	33	56	85	4	48	40	4	44	37	81	59	56	40	96
19	36	7	20	93	11	1	27	69	64	60	75	73	40	72	37	69	90	60	75	93	74	87	76	4	71	72	31	14	47	62
20	32	56	16	17	76	51	31	44	31	96	34	23	3	96	93	61	35	81	28	35	98	57	41	22	70	79	96	84	8	82
21	54	66	13	78	15	70	69	33	53	6	9	94	35	79	64	20	68	81	60	87	50	55	34	66	56	58	58	17	36	28
22	33	80	10	45	75	18	76	51	2	28	100	5	48	5	43	74	99	75	32	95	80	20	26	40	63	48	11	15	65	28
23	54	91	88	61	69	48	73	12	64	7	71	45	17	66	18	82	91	74	53	96	77	20	29	54	35	92	74	10	93	18
24	43	91	32	92	11	5	65	85	19	77	100	63	16	15	44	95	11	71	56	15	4	40	16	34	62	82	88	11	53	11
25	6	64	32	42	73	18	23	73	94	24	76	43	39	66	54	33	69	70	45	23	64	77	61	6	63	27	78	58	10	41
26	94	53	73	14	74	69	87	75	62	95	46	44	78	40	63	2	50	49	46	94	57	49	24	92	87	75	7	25	46	49
27	4	85	71	44	76	85	92	40	59	37	98	97	19	61	39	54	6	55	51	28	33	77	53	84	84	95	50	37	39	88
28	95	29	5	74	15	44	29	97	36	7	58	14	43	11	63	18	80	84	75	76	83	82	12	21	21	5	2	39	63	61
29	77	54	18	17	24	32	3	65	32	91	62	64	74	18	35	75	90	94	20	79	3	79	61	25	39	69	40	52	78	74
30	82	86	30	89	100	80	35	68	94	22	84	27	48	28	44	12	13	28	81	36	8	83	5	28	20	18	46	34	98	18
;


option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 2b:    x_i = j significa che l'elemento i è in posizione j
reset;
param n;
set A :=  {1..n};
param c{A, A};

var x{A} integer >= 1 <= n;
var s{A,A} binary;

minimize z: sum{i in A, j in A} c[i,j]*s[i,j];
s.t. v1{i in A, j in A: i != j}: x[j] - x[i] + 1 <= n*s[i,j];
s.t. v2{i in A, j in A: i != j}: x[i] - x[j] + 1 <= n*(1 - s[i,j]);


data;
param n := 30;
param c :
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30 :=
1	4	23	43	91	57	1	63	47	74	68	15	84	10	28	35	18	53	46	88	67	97	67	91	52	71	75	100	95	37	44
2	16	96	97	50	63	15	22	80	37	16	96	31	57	38	100	13	63	29	43	79	80	47	43	12	15	13	82	97	68	70
3	94	74	50	83	22	70	66	20	100	88	36	47	86	21	21	58	72	84	53	75	100	43	51	93	59	52	55	58	9	16
4	60	33	64	73	81	56	34	74	97	12	57	68	94	85	94	13	88	24	100	77	12	39	3	89	81	49	18	31	94	83
5	11	77	53	30	8	3	78	63	78	2	34	63	65	41	24	56	51	36	2	79	11	82	8	96	1	66	75	16	62	86
6	19	77	23	37	81	78	26	92	12	40	17	74	72	36	19	67	5	57	89	65	17	69	40	8	67	10	71	87	97	40
7	97	84	77	31	51	93	51	95	73	8	92	83	14	73	51	63	91	44	73	48	71	75	76	24	55	42	6	80	56	20
8	92	84	93	38	28	38	33	7	21	100	43	23	6	44	68	52	53	19	73	12	95	1	65	62	27	22	71	6	50	36
9	87	3	10	89	67	78	77	90	72	37	49	6	59	25	84	57	80	26	31	95	83	68	97	47	2	56	42	29	45	47
10	25	37	10	14	41	25	41	65	49	40	67	45	78	47	25	96	2	4	40	65	86	9	44	17	57	42	77	17	59	62
11	6	10	38	90	70	72	74	86	97	82	45	28	97	81	72	26	98	76	64	79	28	30	93	27	95	64	84	53	70	72
12	21	33	90	74	22	74	96	94	50	69	69	54	19	26	24	24	57	10	88	25	66	42	52	28	38	44	89	18	38	32
13	90	48	47	89	34	20	79	69	11	99	57	3	60	89	19	55	99	61	89	66	19	6	85	9	38	52	15	27	37	46
14	76	46	1	71	73	13	23	5	92	20	90	90	55	16	21	46	30	56	14	13	93	33	34	28	9	81	91	35	41	79
15	4	53	50	43	46	42	66	22	30	30	58	4	88	83	44	21	92	38	46	25	48	65	36	29	71	92	99	1	78	76
16	38	54	79	1	78	72	4	83	70	85	67	66	72	86	95	96	32	89	56	60	74	8	53	25	13	97	78	24	6	22
17	81	9	96	19	40	72	1	66	52	80	85	24	17	87	98	21	9	63	63	32	26	57	93	31	78	87	91	37	4	93
18	3	98	25	9	35	57	7	16	65	38	56	43	54	44	55	6	33	56	85	4	48	40	4	44	37	81	59	56	40	96
19	36	7	20	93	11	1	27	69	64	60	75	73	40	72	37	69	90	60	75	93	74	87	76	4	71	72	31	14	47	62
20	32	56	16	17	76	51	31	44	31	96	34	23	3	96	93	61	35	81	28	35	98	57	41	22	70	79	96	84	8	82
21	54	66	13	78	15	70	69	33	53	6	9	94	35	79	64	20	68	81	60	87	50	55	34	66	56	58	58	17	36	28
22	33	80	10	45	75	18	76	51	2	28	100	5	48	5	43	74	99	75	32	95	80	20	26	40	63	48	11	15	65	28
23	54	91	88	61	69	48	73	12	64	7	71	45	17	66	18	82	91	74	53	96	77	20	29	54	35	92	74	10	93	18
24	43	91	32	92	11	5	65	85	19	77	100	63	16	15	44	95	11	71	56	15	4	40	16	34	62	82	88	11	53	11
25	6	64	32	42	73	18	23	73	94	24	76	43	39	66	54	33	69	70	45	23	64	77	61	6	63	27	78	58	10	41
26	94	53	73	14	74	69	87	75	62	95	46	44	78	40	63	2	50	49	46	94	57	49	24	92	87	75	7	25	46	49
27	4	85	71	44	76	85	92	40	59	37	98	97	19	61	39	54	6	55	51	28	33	77	53	84	84	95	50	37	39	88
28	95	29	5	74	15	44	29	97	36	7	58	14	43	11	63	18	80	84	75	76	83	82	12	21	21	5	2	39	63	61
29	77	54	18	17	24	32	3	65	32	91	62	64	74	18	35	75	90	94	20	79	3	79	61	25	39	69	40	52	78	74
30	82	86	30	89	100	80	35	68	94	22	84	27	48	28	44	12	13	28	81	36	8	83	5	28	20	18	46	34	98	18
;

option solver cplex;
option cplex_options "mipdisplay=2 timelimit=120";
solve;

## Problema
- Un robot deve processare 10 pezzi.
- Il robot deve essere attrezzato quando passa da un pezzo al successivo.
- I costi di attrezzaggio sono quelli riportati in tabella.


| |A|B|C|D|E|F|G|H|K|L|
|---|---|---|---|---|---|---|---|---|---|---|
A | 0|	2|	4|	5|	3|	2|	2|	3|	3|	5|
B | 4|	0|	1|	2|	1|	3|	5|	2|	5|	2|
C|2|	3|	0|	2|	5|	2|	1|	5|	3|	3|
D|3|	2|	5|	0|	2|	3|	5|	2|	4|	4|
E|2|	3|	3|	3|	0|	4|	2|	4|	5|	3|
F|2|	1|	2|	5|	2|	0|	1|	2|	3|	1|
G|1|	5|	4|	3|	3|	2|	0|	3|	1|	5|
H|5|	2|	3|	4|	2|	5|	3|	0|	2|	1|
K|3|	1|	4|	2|	5|	3|	4|	2|	0|	2|
L|4|	2|	3|	1|	3|	1|	5|	4|	5|	0|

Esercizio: Come si può esprimere la funzione obiettivo utilizzando una delle alternative discusse in precedenza?

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 1:  x_ij = 1 se i precede (non in senso stretto) j
reset;
param n;
set A :=  {1..n};
param c{A, A};

var x{A,A} binary;

minimize z: sum{i in A, j in A} c[i,j]*x[i,j];
s.t. v1{i in A, j in A: i != j}: x[i,j] + x[j,i] = 1;
s.t. v2{i in A, j in A, k in A: i != j && j != k}: x[i,j] + x[j,k] + x[k,i] <= 2;


data;
param n := 10;
param c : 1 2 3 4 5 6 7 8 9 10 :=
1 0 2 4 5 3 2 2 3 3 5
2 4 0 1 2 1 3 5 2 5 2
3 2 3 0 2 5 2 1 5 3 3
4 3 2 5 0 2 3 5 2 4 4
5 2 3 3 3 0 4 2 4 5 3
6 2 1 2 5 2 0 1 2 3 1
7 1 5 4 3 3 2 0 3 1 5
8 5 2 3 4 2 5 3 0 2 1
9 3 1 4 2 5 3 4 2 0 2
10 4 2 3 1 3 1 5 4 5 0
;

option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval

delete data index;
param index{A};
for{i in A} let index[i] := sum{j in A}x[j,i];

printf "\n\n La sequenza e': ";
for{i in A}{
    for{j in A}{
        if index[j] == i-1 then {printf "%i - ", j; break;}
    }
}
printf " di costo %g \n\n", z;
display x;



 La sequenza e': 6 - 8 - 5 - 3 - 7 - 9 - 10 - 4 - 1 - 2 -  di costo 113 

x [*,*]
:    1   2   3   4   5   6   7   8   9  10    :=
1    0   1   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0
3    1   1   0   1   0   0   1   0   1   1
4    1   1   0   0   0   0   0   0   0   0
5    1   1   1   1   0   0   1   0   1   1
6    1   1   1   1   1   0   1   1   1   1
7    1   1   0   1   0   0   0   0   1   1
8    1   1   1   1   1   0   1   0   1   1
9    1   1   0   1   0   0   0   0   0   1
10   1   1   0   1   0   0   0   0   0   0
;



In [ ]:
%%ampl_eval
expand z;
display x;

Evidentemente anche le alternative 2 e 2b sono di scarsa utilità. In definitiva, nessuna delle precedenti variabili di decisione è utilizzabile **direttamente** per esprimere la funzione obiettivo perchè occorre conteggiare solo i costi di setup tra pezzi **consecutivi** nella sequenza. Introduciamo quindi una nuova variabile:   

### Alternativa 1b

4. $x_{ij} = 1$ se l'elemento $i$ precede **immediatamente** l'elemento $j$.

Benchè la variabile somigli molto a quella definita nella prima alternativa ($x_{ij} = 1$ se l'elemento $i$ precede l'elemento $j$.), non si possono utilizzare gli stessi vincoli. Infatti:

- la condizione $x_{ij} + x_{ji} = 1$ è vera solo per coppie di elementi consecutivi e non per tutte le coppie $i,j$; il vincolo quindi dovrebbe essere rilassato in $x_{ij} + x_{ji} \le 1$ ma così facendo il vettore nullo è soluzione.

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 1b:
model;
reset;
param n;
set A :=  {1..n};
param c{A, A};

var x{A,A} binary;

minimize z: sum{i in A, j in A} c[i,j]*x[i,j];
s.t. v1{i in A, j in A: i != j}: x[i,j] + x[j,i] <= 1;
s.t. v2{i in A, j in A, k in A: i != j && j != k}: x[i,j] + x[j,k] + x[k,i] <= 2;


data;
param n := 10;
param c : 1 2 3 4 5 6 7 8 9 10 :=
1 0 2 4 5 3 2 2 3 3 5
2 4 0 1 2 1 3 5 2 5 2
3 2 3 0 2 5 2 1 5 3 3
4 3 2 5 0 2 3 5 2 4 4
5 2 3 3 3 0 4 2 4 5 3
6 2 1 2 5 2 0 1 2 3 1
7 1 5 4 3 3 2 0 3 1 5
8 5 2 3 4 2 5 3 0 2 1
9 3 1 4 2 5 3 4 2 0 2
10 4 2 3 1 3 1 5 4 5 0
;


option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval
display x;

x [*,*]
:    1   2   3   4   5   6   7   8   9  10    :=
1    0   0   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   0   0
10   0   0   0   0   0   0   0   0   0   0
;



La semantica della nuova variabile $x_{ij}$ può essere definita imponendo che:
- ogni elemento (a parte l'ultimo) abbia esattamente un successore;
- ogni elemento (a parte il primo) abbia esattamente un predecessore;

Siccome non possiamo stabilire a priori chi siano il primo e ultimo elemento della sequenza,  aggiungiamo due elementi fittizi $s$ e $t$ (inizio e fine sequenza) e introduciamo i seguenti vincoli:

ogni elemento (tranne $s$ e compreso $t$) ha **esattamente** un predecessore (eventualmente $s$):
$$
\sum_{i \in A \cup {s}} x_{ij} = 1 \qquad \qquad j \in A \cup {t}     
$$

ogni elemento (tranne $t$ e compreso $s$) ha **esattamente** un successore (eventualmente $t$):
$$
\sum_{j \in A \cup {t}} x_{ij} = 1 \qquad \qquad i \in A \cup {s}     
$$

In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 1b:
model;
reset;

param n;
set A :=  {1..n};
# 0 e 11 sono elementi dummy s e t
set B := {0} union A union {n+1};

param c{A,A} >= 0;

# Dichiarazione delle variabili
var x{B,B} binary;

# Funzione obiettivo
minimize z: sum{i in A, j in A}c[i,j]*x[i,j];

# ogni elemento (eccetto il primo) ha esattamente un predecessore
s.t. predecessori{j in B diff {0}}: sum{i in B diff {n+1}: i != j} x[i,j] = 1;

# ogni elemento (eccetto l'ultimo) ha esattamente un successore
s.t. successori{j in B diff {n+1}}: sum{i in B diff {0}: i != j} x[j,i] = 1;

data;
param n := 10;
param c : 1 2 3 4 5 6 7 8 9 10 :=
1 0 2 4 5 3 2 2 3 3 5
2 4 0 1 2 1 3 5 2 5 2
3 2 3 0 2 5 2 1 5 3 3
4 3 2 5 0 2 3 5 2 4 4
5 2 3 3 3 0 4 2 4 5 3
6 2 1 2 5 2 0 1 2 3 1
7 1 5 4 3 3 2 0 3 1 5
8 5 2 3 4 2 5 3 0 2 1
9 3 1 4 2 5 3 4 2 0 2
10 4 2 3 1 3 1 5 4 5 0
;

option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval
display x;

x [*,*]
:    0   1   2   3   4   5   6   7   8   9  10  11    :=
0    0   0   0   0   0   0   0   0   1   0   0   0
1    0   0   0   0   0   0   1   0   0   0   0   0
2    0   0   0   0   0   1   0   0   0   0   0   0
3    0   0   0   0   0   0   0   1   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0   0   1
5    0   1   0   0   0   0   0   0   0   0   0   0
6    0   0   0   1   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   1   0   0
8    0   0   0   0   0   0   0   0   0   0   1   0
9    0   0   1   0   0   0   0   0   0   0   0   0
10   0   0   0   0   1   0   0   0   0   0   0   0
11   0   0   0   0   0   0   0   0   0   0   0   0
;



In [ ]:
%%ampl_eval
printf "\n\n La sequenza e': 0 - ";

delete data i;
param i;
let i := 0;
repeat{
    for{j in B}{
        if x[i,j] == 1 then {
            printf "%i - ", j;
            let i := j;
        }
    }
}while i < n+1;



 La sequenza e': 0 - 8 - 10 - 4 - 11 - 

**uhm... Cosa succede?**

la soluzione è una sequenza parziale + un ciclo...
- la condizione $x_{ij} + x_{jk} + x_{ki} \le 2$ non garantisce più l'eliminazione di tutti i cicli e bisognerebbe estenderla a tutti i sottoinsiemi di nodi

Occorre introdurre una nuova variabile $y_i$ che indica la posizione dell'elemento $i$-esimo e i seguenti vincoli che evitano la presenza di cicli

$
y_0 = 0
$

$
y_i - y_j + 1 \le |B|(1 - x_{ij}) \qquad \qquad i,j \in B \setminus \{0\}, i \neq j     
$


In [ ]:
%%ampl_eval
##########################  ALTERNATIVA 1b:
model;
reset;

param n;
set A :=  {1..n};
# 0 e 11 sono elementi dummy s e t
set B := {0} union A union {n+1};

param c{A,A} >= 0;


# Dichiarazione delle variabili
var x{B,B} binary;
var y{B} <= card(B), integer;  # posizione dell'elemento nella sequenza

# Funzione obiettivo
minimize z: sum{i in A, j in A}c[i,j]*x[i,j];

# ogni elemento (eccetto 0) ha esattamente un predecessore
s.t. predecessori{j in B diff {0}}: sum{i in B diff {n+1}: i != j} x[i,j] = 1;

# ogni elemento (eccetto 11) ha esattamente un successore
s.t. successori{j in B diff {n+1}}: sum{i in B diff {0}: i != j} x[j,i] = 1;

subject to fix_y1: y[0] = 0;
subject to sequence{i in B, j in B diff {0}: i <> j}: y[i] - y[j] + 1 <= card(B) * (1 - x[i,j]);

data;
param n := 10;
param c : 1 2 3 4 5 6 7 8 9 10 :=
1 0 2 4 5 3 2 2 3 3 5
2 4 0 1 2 1 3 5 2 5 2
3 2 3 0 2 5 2 1 5 3 3
4 3 2 5 0 2 3 5 2 4 4
5 2 3 3 3 0 4 2 4 5 3
6 2 1 2 5 2 0 1 2 3 1
7 1 5 4 3 3 2 0 3 1 5
8 5 2 3 4 2 5 3 0 2 1
9 3 1 4 2 5 3 4 2 0 2
10 4 2 3 1 3 1 5 4 5 0
;

option solver cplex;
option cplex_options "mipdisplay=2";
solve;

CPLEX 22.1.1.0: mipdisplay=2
MIP Presolve eliminated 0 rows and 10 columns.
Reduced MIP has 143 rows, 122 columns, and 564 nonzeros.
Reduced MIP has 111 binaries, 11 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.31 ticks)
Cover probing fixed 0 vars, tightened 20 bounds.
Detecting symmetries...
MIP Presolve eliminated 1 rows and 0 columns.
MIP Presolve modified 22 coefficients.
Reduced MIP has 142 rows, 122 columns, and 562 nonzeros.
Reduced MIP has 111 binaries, 11 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.27 ticks)
Cover probing fixed 0 vars, tightened 10 bounds.
Clique table members: 67.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.23 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                

In [ ]:
%%ampl_eval
display x;

In [ ]:
%%ampl_eval
printf "\n\n La sequenza e': ";

delete data i;
param i;
let i := 0;
repeat{
    for{j in B}{
        if x[i,j] == 1 then {
            printf "%i - ", j;
            let i := j;
        }
    }
}while i <= n;

### Da sequenza a ciclo
Qual è il modello per ottenere il **ciclo** ottimo di produzione?

Siccome in un ciclo non c'è un elemento di inizio e uno di fine, il modello è esattamente lo stesso ma applicato all'insieme originale $A$, quindi senza introdurre elementi iniziali e finali fittizi.

ogni elemento ha **esattamente** un predecessore:
$$
\sum_{i \in A} x_{ij} = 1 \qquad \qquad j \in A     
$$

ogni elemento ha **esattamente** un successore:
$$
\sum_{j \in A} x_{ij} = 1 \qquad \qquad i \in A     
$$

In [ ]:
%%ampl_eval
reset;

param n;
set A :=  {1..n};

param c{A,A} >= 0;

# Dichiarazione delle variabili
var x{A,A} binary;
var y{A} <= card(A), integer;

# Funzione obiettivo
minimize z: sum{i in A, j in A}c[i,j]*x[i,j];

# ogni elemento ha esattamente un predecessore
s.t. predecessori{j in A}: sum{i in A: i != j} x[i,j] = 1;

# ogni elemento ha esattamente un successore
s.t. successori{j in A}: sum{i in A: i != j} x[j,i] = 1;

subject to fix_y1: y[1] = 0;
subject to sequence{i in A, j in A diff {1}: i <> j}: y[i] - y[j] + 1 <= card(A) * (1 - x[i,j]);


data;
param n := 10;
param c : 1 2 3 4 5 6 7 8 9 10 :=
1 0 2 4 5 3 2 2 3 3 5
2 4 0 1 2 1 3 5 2 5 2
3 2 3 0 2 5 2 1 5 3 3
4 3 2 5 0 2 3 5 2 4 4
5 2 3 3 3 0 4 2 4 5 3
6 2 1 2 5 2 0 1 2 3 1
7 1 5 4 3 3 2 0 3 1 5
8 5 2 3 4 2 5 3 0 2 1
9 3 1 4 2 5 3 4 2 0 2
10 4 2 3 1 3 1 5 4 5 0
;

option solver cplex;
option cplex_options "mipdisplay=2";
solve;

In [ ]:
%%ampl_eval
display x;
printf "\n\n Il ciclo e':  ";

delete data i;
param i;
let i := 1;
for{1..n}{
    for{j in A}{
        if x[i,j] == 1 then {
            printf "%i - ", j;
            let i := j;
            break;
        }
    }
}

**ESERCIZIO**

Modificare opportunamente i valori della matrice di setup dell'esempio precedente in modo che il ciclo ottimo sia diverso dalla sequenza ottima